#### Import moduldes

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

pd.set_option('display.max_columns', None)

#### Load the data provided

In [ ]:
Train = pd.read_csv("Train.csv")

In [ ]:
Train.head()

In [ ]:
Test = pd.read_csv("Test.csv")

In [ ]:
Test.head()

In [ ]:
game_statistics = pd.read_csv("game_statistics.csv")

In [ ]:
game_statistics.head()

In [ ]:
Train.shape, Test.shape, game_statistics.shape

### Data EDA and Visualisation.

- Lets analyze the data provided by anaswering some few questions

#### How many seasons are in train and test ?

In [ ]:
sns.countplot(Train["Season"])

In [ ]:
sns.countplot(Test["Season"])

- We can see that 2 seasons are provided in the training data while one season is provided in the the test

#### How many unique match IDs are provided?

- From a football perspective, can you find out what's a match, are matches specific to every season?

In [ ]:
Train.Match_ID.nunique()

In [ ]:
Test.Match_ID.nunique()

#### How many games were played in train seasons and test seasons?

In [ ]:
Train.Game_ID.nunique()

In [ ]:
Test.Game_ID.nunique()

#### How many teams were away and home?

In [ ]:
Train["Home Team"].nunique()

In [ ]:
Train["Away Team"].nunique()

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(Train["Home Team"])

In [ ]:
Train["Away Team"].nunique()

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(Train["Away Team"])

In [ ]:
Test["Home Team"].nunique()

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(Test["Home Team"])

In [ ]:
Test["Away Team"].nunique()

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(Test["Away Team"])

- We can see that the number of teams in Home and away are equal in both cases, however the test count of the teams is more balance than the train, could this be as a result of 1 season in test and 2 seasons in the train? For the train can you countplot per season?

#### Did the team Win, Lose or Draw?

- Let's look at the target column, the reason for creating our machine leaning model, the goal of every footbal game is to determine the winner

In [ ]:
Train["Score"].nunique()

In [ ]:
sns.countplot(Train["Score"])

We have 3 classifications, did the away team win, did both teams draw or did the home team win? We can see that in most cases the home team won. There's much more you can do with this field:

    - Look at every team, the number of games played? did they win or lose, are most of their wins during home or away game?

- **You can do more EDAs on the game statistics file**

### Feature Engineering and Modelling

#### Game statistics 

- This file shows how the game was played from start to end, the actions of the game, which player was it passed to etc.
- We will need to merge this with the train and test data provided.

In [ ]:
Train["Train"] = 1
Test["Train"]= 0

train_test = pd.concat([Train, Test])

In [ ]:
train_test.shape

In [ ]:
train_test.head(3)

In [ ]:
game_statistics.head(3)

In [ ]:
all_data = game_statistics.merge(train_test, on="Game_ID")

In [ ]:
all_data.shape

In [ ]:
all_data.head()

#### How to deal with column that are categorical?

- There are various ways of doing this:-

    - One hot encoding 
    - label encoding etc
    
    
- In this starter nb, I'll use label encoding, you can try other other ways

In [ ]:
le = preprocessing.LabelEncoder()

all_data["Game_ID"] = le.fit_transform(all_data["Game_ID"])
game_mapping = dict(zip(le.classes_, range(len(le.classes_))))
all_data["Player_ID"] = le.fit_transform(all_data["Player_ID"])

- For the teams ensure they are transformed the same, you do not want a scenario where team Circinus is mapped as 1 on one column and mapped as 2 on another column, this will be inconsistent and will affect the model results

In [ ]:
all_data["Team"] = le.fit_transform(all_data["Team"])

mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [ ]:
mapping

In [ ]:
all_data["Opposition_Team"] = all_data["Opposition_Team"].apply(lambda x: mapping[x])
all_data["Home Team"] = all_data["Home Team"].apply(lambda x: mapping[x])
all_data["Away Team"] = all_data["Away Team"].apply(lambda x: mapping[x])

In [ ]:
all_data.head()

In [ ]:
all_data.isnull().sum()

In [ ]:
#all_data["Action"] = le.fit_transform(all_data["Action"])
all_data["Half"] = le.fit_transform(all_data["Half"].astype(str))
all_data["Manager"] = le.fit_transform(all_data["Manager"].astype(str))

#df2 = df2.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

In [ ]:
all_data.head()

- For this nb, I'll drop most of the columns that have alot on NAN values, you can choose to fill those NAN values since the columns might be worth including in the modeling

In [ ]:
all_data.drop(["next_player", "next_action", "next_x", "next_y", "event_id", "next_team",
               "next_event_id", "xt_value", "Date"], axis=1, inplace=True)

In [ ]:
all_data.head()

In [ ]:
Train =all_data[all_data["Train"] == 1]
Train.drop("Train", axis=1, inplace=True)
Test = all_data[all_data["Train"] == 0]
Test.drop(["Train", "Score"], axis=1, inplace=True)
Test = Test.reset_index(drop=True)

In [ ]:
Train.shape, Test.shape

In [ ]:
Train.head(2)

In [ ]:
Test.head(2)

In [ ]:
# to avoid nans, will map score onli in the training set
Train["Score"] = le.fit_transform(Train["Score"])
score_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [ ]:
score_mapping

In [ ]:
Train.isnull().sum()

In [ ]:
Train.drop(["Match_ID","id","Season"], axis=1, inplace = True)
Train.head()

In [ ]:
X = Train.drop(["Team","Opposition_Team","Unnamed: 0","Game_ID","Score"], axis=1)
y = Train["Score"]

clf = LogisticRegression(multi_class='ovr', max_iter=1000)
clf.fit(X, y)

In [ ]:
Test.isnull().sum()

In [ ]:
Test.drop(["Match_ID","Season","id"], axis=1, inplace = True)

In [ ]:
Test = Test.fillna(Test[['Shots','SoT','Accurate passes','Inaccurate passes','Passes','Start_minutes']].mean())

In [ ]:
Test.isnull().sum()

In [ ]:
test_predictions  = clf.predict(Test.drop(["Team","Opposition_Team","Unnamed: 0","Game_ID"],axis=1))

In [ ]:
Test["Score"] = test_predictions

In [ ]:
Test["Score"].unique()

In [ ]:
score_mapping = { 'Draw': 1, 'Home Win': 2, 'Away win': 0}
score_mapping

In [ ]:
score_mapping = {value:key for key, value in score_mapping.items()}
Test["Score"] = Test["Score"].apply(lambda x: score_mapping[x])

In [ ]:
game_mapping = {'ID_00EFNL7L': 0,
 'ID_00J124MI': 1,
 'ID_03VRW8WV': 2,
 'ID_04T9W310': 3,
 'ID_05YW7A1R': 4,
 'ID_064HV3UA': 5,
 'ID_06QXUEYD': 6,
 'ID_06SV1XSM': 7,
 'ID_08N8KYTM': 8,
 'ID_0A2WEVY3': 9,
 'ID_0BFR12C0': 10,
 'ID_0D97CTM2': 11,
 'ID_0DQCXCXH': 12,
 'ID_0H4KVTOZ': 13,
 'ID_0M5K2L1J': 14,
 'ID_0O3OGCB9': 15,
 'ID_0OPXV0TY': 16,
 'ID_0OR30CC4': 17,
 'ID_0OW7LBXH': 18,
 'ID_0PBGEMAI': 19,
 'ID_0SXG1LVI': 20,
 'ID_0TIEZ7MG': 21,
 'ID_0UB8ADG9': 22,
 'ID_12945FWD': 23,
 'ID_133YXY0T': 24,
 'ID_172V2IXW': 25,
 'ID_1B6FP9H5': 26,
 'ID_1D1X4PFB': 27,
 'ID_1ED799VW': 28,
 'ID_1FEH4NFI': 29,
 'ID_1IQNQPAW': 30,
 'ID_1J98M9UL': 31,
 'ID_1JGSZL4I': 32,
 'ID_1KCBWUKE': 33,
 'ID_1KKTGS75': 34,
 'ID_1LEM1AAP': 35,
 'ID_1TGCIK70': 36,
 'ID_1Y735BT9': 37,
 'ID_1YF71544': 38,
 'ID_1ZPA9XMX': 39,
 'ID_20BPYE4G': 40,
 'ID_21BTFEL5': 41,
 'ID_21SVBXLQ': 42,
 'ID_22R4A4OV': 43,
 'ID_24FZH22W': 44,
 'ID_24HKPKQH': 45,
 'ID_25GTI42K': 46,
 'ID_27YV4RRO': 47,
 'ID_2947LQTO': 48,
 'ID_2B96PJLU': 49,
 'ID_2BHM1MAF': 50,
 'ID_2DMNO9QP': 51,
 'ID_2E3951AN': 52,
 'ID_2FUAPJ5S': 53,
 'ID_2KH2XHCH': 54,
 'ID_2QNA7OQE': 55,
 'ID_2QSZ4K64': 56,
 'ID_2V7QGMM2': 57,
 'ID_2WSDLFNM': 58,
 'ID_2X3AC0BS': 59,
 'ID_2ZK5TE39': 60,
 'ID_313CQJ6L': 61,
 'ID_33AOLMD8': 62,
 'ID_34HDKVFW': 63,
 'ID_35M6T9JZ': 64,
 'ID_36NZ9IU5': 65,
 'ID_38GWYHWI': 66,
 'ID_39S61786': 67,
 'ID_3CWKIG2G': 68,
 'ID_3I1STYJX': 69,
 'ID_3LDF38RP': 70,
 'ID_3NW5KUQ3': 71,
 'ID_3SHIZPTH': 72,
 'ID_3SIVKWL4': 73,
 'ID_3TIWWKU9': 74,
 'ID_3WQ0X2RV': 75,
 'ID_40BVMU42': 76,
 'ID_40OQTBJ8': 77,
 'ID_42DWN5QA': 78,
 'ID_42WQ2WIV': 79,
 'ID_45M9BNQO': 80,
 'ID_46UP27C8': 81,
 'ID_4CYGD80A': 82,
 'ID_4G117ZRE': 83,
 'ID_4KXP34GC': 84,
 'ID_4LTRF5VB': 85,
 'ID_4M8BH80D': 86,
 'ID_4PB4VL5Q': 87,
 'ID_4Q60CD7W': 88,
 'ID_4VQVJS1O': 89,
 'ID_4WLLNCYX': 90,
 'ID_4X3QDC8P': 91,
 'ID_530FN5PE': 92,
 'ID_568NZHSC': 93,
 'ID_56ZZJ9EI': 94,
 'ID_58IZEI5C': 95,
 'ID_58Y3YYGP': 96,
 'ID_5H9QBQMQ': 97,
 'ID_5HWA04IU': 98,
 'ID_5K4BCEUJ': 99,
 'ID_5KHVZBQE': 100,
 'ID_5KQZOTHQ': 101,
 'ID_5LFD56QF': 102,
 'ID_5RTFVRBT': 103,
 'ID_5X8VFC64': 104,
 'ID_5XNJBBBU': 105,
 'ID_5Y6WIRCG': 106,
 'ID_60EADYBI': 107,
 'ID_628MNEH9': 108,
 'ID_65XZO9R3': 109,
 'ID_66IWBNXT': 110,
 'ID_68FWDT6H': 111,
 'ID_6DCVCZN8': 112,
 'ID_6IIQ4TZD': 113,
 'ID_6K7LRVTC': 114,
 'ID_6KO0SC5W': 115,
 'ID_6NMJFGI6': 116,
 'ID_72XLLS9T': 117,
 'ID_7DM8G9SL': 118,
 'ID_7DRHHPZK': 119,
 'ID_7GHLYGYV': 120,
 'ID_7H9RUYR2': 121,
 'ID_7HNMFC11': 122,
 'ID_7HRBFGSB': 123,
 'ID_7HWNZT53': 124,
 'ID_7JTHUMTF': 125,
 'ID_7KQ7M6CR': 126,
 'ID_7MS32IW5': 127,
 'ID_7MWJO1RG': 128,
 'ID_7Q5V45IR': 129,
 'ID_7QNNI0YQ': 130,
 'ID_7RVSY2WQ': 131,
 'ID_7SH1NGU4': 132,
 'ID_7XQJ4ZA4': 133,
 'ID_7YEZ3VK0': 134,
 'ID_7ZBSNKA9': 135,
 'ID_7ZQ7E7Q2': 136,
 'ID_826SG25P': 137,
 'ID_8518U587': 138,
 'ID_85W3ZWMS': 139,
 'ID_86GTEWAA': 140,
 'ID_897S67BZ': 141,
 'ID_8AKT073X': 142,
 'ID_8CRFDQZA': 143,
 'ID_8DEUQQUO': 144,
 'ID_8ITGS2P3': 145,
 'ID_8JROBJ91': 146,
 'ID_8JTQDHX3': 147,
 'ID_8LEIJZJI': 148,
 'ID_8MCII6F1': 149,
 'ID_8O8VRN8A': 150,
 'ID_8QTW09GB': 151,
 'ID_8W85GO7G': 152,
 'ID_8ZJMCSAG': 153,
 'ID_96T3LR7R': 154,
 'ID_9AKIA9XC': 155,
 'ID_9DJ219JY': 156,
 'ID_9DPBXX3W': 157,
 'ID_9FMP4X22': 158,
 'ID_9H3FH940': 159,
 'ID_9JEI8DWV': 160,
 'ID_9KVVOCR6': 161,
 'ID_9NG7M7IH': 162,
 'ID_9NX3Y6RK': 163,
 'ID_9P65QCMQ': 164,
 'ID_9QKWORXG': 165,
 'ID_9UVQHS74': 166,
 'ID_9V68FI7L': 167,
 'ID_9V6TAYFV': 168,
 'ID_9XCMP096': 169,
 'ID_A0EALXUP': 170,
 'ID_A1ARNO3H': 171,
 'ID_A1PN23J6': 172,
 'ID_A2FD42KH': 173,
 'ID_A46DVQWP': 174,
 'ID_A55NG93L': 175,
 'ID_A5U63MGQ': 176,
 'ID_A84IUATV': 177,
 'ID_AB2XN1F5': 178,
 'ID_AFHJCI8U': 179,
 'ID_AJQ6AXK8': 180,
 'ID_ANHOSW6K': 181,
 'ID_ANK9DPJS': 182,
 'ID_AO08Y8FZ': 183,
 'ID_AO17ZS96': 184,
 'ID_AOLHJTW3': 185,
 'ID_APRR2449': 186,
 'ID_AQD3YVMC': 187,
 'ID_AT1RY70Q': 188,
 'ID_B32VWKSX': 189,
 'ID_B3NVY9HP': 190,
 'ID_B5ED50XX': 191,
 'ID_B8MME8GP': 192,
 'ID_B8ZVJSX7': 193,
 'ID_B9JSESNH': 194,
 'ID_BBYSJZ7Z': 195,
 'ID_BJ2VX0EX': 196,
 'ID_BJHEDY9T': 197,
 'ID_BK1X6A61': 198,
 'ID_BOI7CXJV': 199,
 'ID_BR430MM1': 200,
 'ID_BXLQY78T': 201,
 'ID_C1EA08ZR': 202,
 'ID_C28GUZGL': 203,
 'ID_C51DS9XE': 204,
 'ID_CAO3DHF3': 205,
 'ID_CEB18NBQ': 206,
 'ID_CFNRNH31': 207,
 'ID_CGFDXN9S': 208,
 'ID_CGX3THAU': 209,
 'ID_CHH8Y1BD': 210,
 'ID_CNPU22AH': 211,
 'ID_CT02DQO5': 212,
 'ID_CV7BOXYD': 213,
 'ID_CV9VOLIU': 214,
 'ID_CVL86B7P': 215,
 'ID_CW6K7CPC': 216,
 'ID_D1BKOMNB': 217,
 'ID_D2JPKHY3': 218,
 'ID_D4BBONMS': 219,
 'ID_D4WOYK9B': 220,
 'ID_D801VVTI': 221,
 'ID_D8RLSTH5': 222,
 'ID_DBF91Z88': 223,
 'ID_DC2OE4J0': 224,
 'ID_DEBRT4C8': 225,
 'ID_DF19AH9G': 226,
 'ID_DKEDN070': 227,
 'ID_DKNFRR94': 228,
 'ID_DKYRY1VS': 229,
 'ID_DL7257PH': 230,
 'ID_DLPZZFIJ': 231,
 'ID_DN9ALJT4': 232,
 'ID_DPPFAJ6O': 233,
 'ID_DSZ1VC2A': 234,
 'ID_DTTQRNE4': 235,
 'ID_DVUF7IYI': 236,
 'ID_DXFDO3HX': 237,
 'ID_DYAVEIYE': 238,
 'ID_E4NAVOHG': 239,
 'ID_E7000EMK': 240,
 'ID_EEAWBOYL': 241,
 'ID_EETOBLMQ': 242,
 'ID_EFUM1IB3': 243,
 'ID_EIIQ4HJL': 244,
 'ID_EK4VZ52Q': 245,
 'ID_EKI17JGO': 246,
 'ID_EMIAV5PV': 247,
 'ID_ENJWVMFU': 248,
 'ID_ENOCMDPI': 249,
 'ID_EOIECRE2': 250,
 'ID_EP02V6O9': 251,
 'ID_EU409IWG': 252,
 'ID_EVSAW1BF': 253,
 'ID_EVTW8X2J': 254,
 'ID_EWEN4C0N': 255,
 'ID_EWFWK6L1': 256,
 'ID_EWTPLPQM': 257,
 'ID_F04XYXNO': 258,
 'ID_F19EBYGB': 259,
 'ID_F1CNSJBO': 260,
 'ID_F2M9J1QJ': 261,
 'ID_F60AZ116': 262,
 'ID_FDCEHVOD': 263,
 'ID_FEOI64KN': 264,
 'ID_FH7XN5HK': 265,
 'ID_FHG03TE0': 266,
 'ID_FKNOHQ2F': 267,
 'ID_FRUGDE4G': 268,
 'ID_FWJMLGFB': 269,
 'ID_G2O8B9R9': 270,
 'ID_G4CN5BNX': 271,
 'ID_G5EFF75Z': 272,
 'ID_G78FOQYQ': 273,
 'ID_G87HE5T9': 274,
 'ID_G91KCS98': 275,
 'ID_GBP5RWBW': 276,
 'ID_GGFY5Z38': 277,
 'ID_GGRLN92Z': 278,
 'ID_GI90U9RD': 279,
 'ID_GKU067BE': 280,
 'ID_GLACPTCA': 281,
 'ID_GOIGEZE8': 282,
 'ID_GOMKISKS': 283,
 'ID_GP3WNMQH': 284,
 'ID_GPZBRVUQ': 285,
 'ID_GU6BX7OY': 286,
 'ID_GUGKLLI9': 287,
 'ID_GVX36W01': 288,
 'ID_H0Q7JNV1': 289,
 'ID_H27PCYW4': 290,
 'ID_H49BIKG7': 291,
 'ID_H4WRH27A': 292,
 'ID_H5QG5VMD': 293,
 'ID_H6H66KH2': 294,
 'ID_H6V0DHCM': 295,
 'ID_HB9WSKP9': 296,
 'ID_HCCBAPTP': 297,
 'ID_HHF8W6GO': 298,
 'ID_HNY0SOAW': 299,
 'ID_HPTFSC3K': 300,
 'ID_HPYKEW7R': 301,
 'ID_HQC5KGL5': 302,
 'ID_HRER0KI5': 303,
 'ID_HT3DTAG1': 304,
 'ID_HTSWMY3T': 305,
 'ID_HV2F2U3U': 306,
 'ID_HYJCUA56': 307,
 'ID_HZKCT0PI': 308,
 'ID_I14RAYEM': 309,
 'ID_I7AY16R0': 310,
 'ID_IBRGSBOA': 311,
 'ID_ICUXU3S2': 312,
 'ID_IDEQCVEC': 313,
 'ID_IDJ1L6GL': 314,
 'ID_IG3ZU2ZF': 315,
 'ID_IGWZ8LQR': 316,
 'ID_II7DWVQR': 317,
 'ID_ILDWDPY2': 318,
 'ID_IM6OHCK8': 319,
 'ID_IPJJKELT': 320,
 'ID_IPWDET7J': 321,
 'ID_IS54QGW9': 322,
 'ID_IS5PC7WP': 323,
 'ID_IUV7FU2U': 324,
 'ID_IXRGB6OV': 325,
 'ID_IZ8B4OAH': 326,
 'ID_IZLDEI78': 327,
 'ID_J1010ZTT': 328,
 'ID_J15AKP1H': 329,
 'ID_J5WBKEM3': 330,
 'ID_J9I1S6T7': 331,
 'ID_JA9L72HB': 332,
 'ID_JFIU2R4K': 333,
 'ID_JFK7XPYK': 334,
 'ID_JGPD54X5': 335,
 'ID_JHECYNWP': 336,
 'ID_JKJP61NL': 337,
 'ID_JL0A7NYU': 338,
 'ID_JMF7GIUY': 339,
 'ID_JMKNLEUZ': 340,
 'ID_JN1VLXHD': 341,
 'ID_JNE2K1SS': 342,
 'ID_JP06JI0C': 343,
 'ID_JW43FR1J': 344,
 'ID_JXNNMUJK': 345,
 'ID_JYOCIVGT': 346,
 'ID_JZ6N5FRO': 347,
 'ID_K06LVF2G': 348,
 'ID_K3VPZMW2': 349,
 'ID_K7L28JTT': 350,
 'ID_K81P666R': 351,
 'ID_K830M6NE': 352,
 'ID_KAG4KAE9': 353,
 'ID_KAHZNQZ4': 354,
 'ID_KC53ZIHC': 355,
 'ID_KD5JPIY4': 356,
 'ID_KDY6XYY0': 357,
 'ID_KFK1U0BS': 358,
 'ID_KJBPWWIH': 359,
 'ID_KKRFBDM9': 360,
 'ID_KL74YZ1W': 361,
 'ID_KLAHLZBN': 362,
 'ID_KLXJD2ZC': 363,
 'ID_KR88Y90B': 364,
 'ID_KSFJ2RGK': 365,
 'ID_KSKX84HK': 366,
 'ID_KST7PRMU': 367,
 'ID_KXGJ4N09': 368,
 'ID_KYEZ4Q5U': 369,
 'ID_KYVAXN0B': 370,
 'ID_L1OMXXIV': 371,
 'ID_L3GUMYJ4': 372,
 'ID_L76JBE84': 373,
 'ID_L80OJ6QI': 374,
 'ID_LCUC9FYE': 375,
 'ID_LEBJ75LN': 376,
 'ID_LEL3JT8E': 377,
 'ID_LJF4GEZC': 378,
 'ID_LL4GFXCN': 379,
 'ID_LRWZVXJZ': 380,
 'ID_LTB6HGNO': 381,
 'ID_LTKK72S4': 382,
 'ID_LUDJ2HBX': 383,
 'ID_LVP6C9HW': 384,
 'ID_LWI3010X': 385,
 'ID_LXRTEQO2': 386,
 'ID_M7QACJDH': 387,
 'ID_M840FM1K': 388,
 'ID_MI4W134C': 389,
 'ID_MJ8THLEX': 390,
 'ID_MM9562CN': 391,
 'ID_MNGV4C4L': 392,
 'ID_MP4ONP0P': 393,
 'ID_MPA7FT70': 394,
 'ID_MUYK98UO': 395,
 'ID_MWCIADS6': 396,
 'ID_MWHDDOC0': 397,
 'ID_MWRHCSMW': 398,
 'ID_MWXN4B53': 399,
 'ID_MXDJNYS9': 400,
 'ID_MZRCNBAQ': 401,
 'ID_N0UG6G1F': 402,
 'ID_N4FNUPW8': 403,
 'ID_N6FOPL2Y': 404,
 'ID_NEG8CDCD': 405,
 'ID_NER74N7H': 406,
 'ID_NFN5I5XU': 407,
 'ID_NFRZQ99K': 408,
 'ID_NMRV6S78': 409,
 'ID_NO60D4TO': 410,
 'ID_NQGKKQRG': 411,
 'ID_NQK2S2JO': 412,
 'ID_NQX0L8NJ': 413,
 'ID_NXCBXYUJ': 414,
 'ID_O1P9FGN4': 415,
 'ID_O30PXNS2': 416,
 'ID_O3EQ8WEA': 417,
 'ID_O4H6XGB4': 418,
 'ID_O50PFEVT': 419,
 'ID_O7S3MLNP': 420,
 'ID_OB71L9F1': 421,
 'ID_OBS64UDF': 422,
 'ID_OBSKQ4V1': 423,
 'ID_OC055HAA': 424,
 'ID_OC87DVM9': 425,
 'ID_ODJJ0THO': 426,
 'ID_OE2XOG04': 427,
 'ID_OHS6RR07': 428,
 'ID_OIZQHOLE': 429,
 'ID_OJUD1I32': 430,
 'ID_OKHY3GBQ': 431,
 'ID_OLMRL7BS': 432,
 'ID_OM37A8LR': 433,
 'ID_ON6TBDP7': 434,
 'ID_OQM43U7K': 435,
 'ID_OT3MQDZC': 436,
 'ID_OW9Z59TM': 437,
 'ID_OWJBAFBQ': 438,
 'ID_OZKQYKQF': 439,
 'ID_P0JSAPAH': 440,
 'ID_P98LF0O1': 441,
 'ID_P9QCOLI6': 442,
 'ID_PBG2XS57': 443,
 'ID_PBON8UC5': 444,
 'ID_PD70LTRE': 445,
 'ID_PDW4BNW1': 446,
 'ID_PGGWBZ29': 447,
 'ID_PJ53WB0E': 448,
 'ID_PK1XYY2J': 449,
 'ID_PK7PTLYZ': 450,
 'ID_PMX7KBOK': 451,
 'ID_PO6SP4VA': 452,
 'ID_PREVA6HZ': 453,
 'ID_PSLB87BL': 454,
 'ID_PYMMJPRC': 455,
 'ID_PYUUJ5M3': 456,
 'ID_Q05IUX5U': 457,
 'ID_Q0QMD9X9': 458,
 'ID_Q3TLZ0NB': 459,
 'ID_Q4V1C0YI': 460,
 'ID_Q5C5K59A': 461,
 'ID_QAJGGM73': 462,
 'ID_QCQ20X8Y': 463,
 'ID_QEHB6SP5': 464,
 'ID_QEMTT0ST': 465,
 'ID_QGNRR2WQ': 466,
 'ID_QHZJNHVX': 467,
 'ID_QIS7V5IV': 468,
 'ID_QIXKBR86': 469,
 'ID_QKQ00O16': 470,
 'ID_QKWD8147': 471,
 'ID_QMJBCJTO': 472,
 'ID_QNZG9PEU': 473,
 'ID_QPA4T2QH': 474,
 'ID_QQYS31OV': 475,
 'ID_QR4DQRG1': 476,
 'ID_QRQCZ4Z4': 477,
 'ID_QV0XBIAB': 478,
 'ID_QVW81QWH': 479,
 'ID_QWUGRXJI': 480,
 'ID_QZLKZJ16': 481,
 'ID_QZMTM1LI': 482,
 'ID_R3I7GSLU': 483,
 'ID_R618L63O': 484,
 'ID_R6C3GV9H': 485,
 'ID_R6SFPFV4': 486,
 'ID_R803GAI9': 487,
 'ID_R9WWGZ6K': 488,
 'ID_RAT7AK8R': 489,
 'ID_RFU5FMC8': 490,
 'ID_RJZDWDAU': 491,
 'ID_RNR88WYL': 492,
 'ID_ROVPAHRR': 493,
 'ID_RPNHBU1X': 494,
 'ID_RROBJ38Y': 495,
 'ID_RRYKCVJW': 496,
 'ID_RT5HR95R': 497,
 'ID_RX2CNZIS': 498,
 'ID_S2DH4W65': 499,
 'ID_S2N3NDJS': 500,
 'ID_S2ZLOYQU': 501,
 'ID_S6RUK5NW': 502,
 'ID_S7T4YYCU': 503,
 'ID_SA37C0V7': 504,
 'ID_SGHWIP1E': 505,
 'ID_SQB73VYU': 506,
 'ID_SREWU5C6': 507,
 'ID_SSBUAMO7': 508,
 'ID_STX8TIE2': 509,
 'ID_SXSLZSI1': 510,
 'ID_SY9KA6L3': 511,
 'ID_T12UUJG7': 512,
 'ID_T1Q3B3GG': 513,
 'ID_T31AU81S': 514,
 'ID_T4Q5NGSF': 515,
 'ID_T5467PO8': 516,
 'ID_T62C8AP0': 517,
 'ID_T64ZJ5ZU': 518,
 'ID_T7H1XN5Z': 519,
 'ID_T7W4WKY9': 520,
 'ID_T9ASGHM0': 521,
 'ID_TEHN7MG2': 522,
 'ID_TFI6B51W': 523,
 'ID_THU0JYSJ': 524,
 'ID_TLXXQLYC': 525,
 'ID_TN3U4DC4': 526,
 'ID_TR5M4KQV': 527,
 'ID_TRN20YF5': 528,
 'ID_TRXZ1689': 529,
 'ID_U2DPW95R': 530,
 'ID_U3SD7K1V': 531,
 'ID_U8OKJQV2': 532,
 'ID_UATYFRPS': 533,
 'ID_UC7PMJ25': 534,
 'ID_UCWLCAW5': 535,
 'ID_UFDKE17M': 536,
 'ID_UGM73VHI': 537,
 'ID_UHMPXW5E': 538,
 'ID_UIJNWDX2': 539,
 'ID_UKIMWWUK': 540,
 'ID_UL4NP2IJ': 541,
 'ID_UL9DKAH4': 542,
 'ID_ULFMIS9R': 543,
 'ID_UMASWHEF': 544,
 'ID_UOTGKHS3': 545,
 'ID_UUQYAGBP': 546,
 'ID_UWJ3YGHF': 547,
 'ID_UXJDXX9L': 548,
 'ID_UXUK6H9F': 549,
 'ID_UY24QO9Y': 550,
 'ID_V891OYVP': 551,
 'ID_V8L2BWZV': 552,
 'ID_VACHXR44': 553,
 'ID_VAG7G5IC': 554,
 'ID_VEHWAM3N': 555,
 'ID_VLSQNHJS': 556,
 'ID_VP9Q1486': 557,
 'ID_VTVBHCO7': 558,
 'ID_VU4MWMEQ': 559,
 'ID_VUN0XPN1': 560,
 'ID_VVG172LB': 561,
 'ID_VZDE4BA3': 562,
 'ID_W1EF4MOT': 563,
 'ID_W25SLQ7X': 564,
 'ID_W3F61OZ8': 565,
 'ID_W5BFF5QC': 566,
 'ID_WASR8LFE': 567,
 'ID_WDA74NYV': 568,
 'ID_WDL1IC3Y': 569,
 'ID_WGM9ZTJ3': 570,
 'ID_WH4S1N9Q': 571,
 'ID_WJ12QQ71': 572,
 'ID_WJIU8X3N': 573,
 'ID_WOXKZITI': 574,
 'ID_WPAZXG4Y': 575,
 'ID_WQ6K1M6J': 576,
 'ID_WRQ0YFYZ': 577,
 'ID_WT3PHABY': 578,
 'ID_WVFRCW5F': 579,
 'ID_WX5LMCE5': 580,
 'ID_WX90Q0Q3': 581,
 'ID_WZASJ4IV': 582,
 'ID_X5UEAAIY': 583,
 'ID_X9050P7D': 584,
 'ID_XBT7BBZD': 585,
 'ID_XCHM541T': 586,
 'ID_XDMKT9HU': 587,
 'ID_XEI81LPI': 588,
 'ID_XFQO60PZ': 589,
 'ID_XIRRA7RV': 590,
 'ID_XJ2WUU6R': 591,
 'ID_XJWVEY79': 592,
 'ID_XK39LD7P': 593,
 'ID_XMM4Z5EZ': 594,
 'ID_XMQ5NR2G': 595,
 'ID_XNS4KRCK': 596,
 'ID_XOSYZEG8': 597,
 'ID_XQGYMP2Q': 598,
 'ID_XQMJ05QY': 599,
 'ID_XVGLDCOL': 600,
 'ID_XW18HGR9': 601,
 'ID_XWI57Q9K': 602,
 'ID_XWL7CYH2': 603,
 'ID_XWV89SWE': 604,
 'ID_Y1FWWSBP': 605,
 'ID_Y1N9XWDE': 606,
 'ID_Y86ZTX6M': 607,
 'ID_YG9IONLS': 608,
 'ID_YGN8TPQ5': 609,
 'ID_YLIXN9AV': 610,
 'ID_YMK5WFEQ': 611,
 'ID_YO3TBJ4R': 612,
 'ID_YO6NFBHZ': 613,
 'ID_YP3N91A2': 614,
 'ID_YQ6DCSJJ': 615,
 'ID_YSOEYNCU': 616,
 'ID_YU30VY06': 617,
 'ID_YVMAC2WR': 618,
 'ID_YXXCNK2G': 619,
 'ID_YYCO8ZBA': 620,
 'ID_Z669YURL': 621,
 'ID_Z71PIKP3': 622,
 'ID_Z8TGBH41': 623,
 'ID_Z8YAZGAP': 624,
 'ID_ZD3UQK1K': 625,
 'ID_ZE3J281E': 626,
 'ID_ZE7D4JAP': 627,
 'ID_ZF8TL0GP': 628,
 'ID_ZIMAODWU': 629,
 'ID_ZJ5I8APD': 630,
 'ID_ZJC5BV5Z': 631,
 'ID_ZL8511XW': 632,
 'ID_ZLWVE54E': 633,
 'ID_ZMO6O1V6': 634,
 'ID_ZOW0PTN6': 635,
 'ID_ZP3MIG6N': 636,
 'ID_ZQOB9ZQL': 637,
 'ID_ZRZC49M1': 638,
 'ID_ZTDZNU6I': 639,
 'ID_ZUPS2JD2': 640,
 'ID_ZWD4FQ05': 641,
 'ID_ZWJJP6T6': 642,
 'ID_ZWPPN4W1': 643,
 'ID_ZYI3C0HF': 644,
 'ID_ZZKYYPVY': 645}


In [ ]:
game_mapping = {value:key for key, value in game_mapping.items()}
Test["Game_ID"] = Test["Game_ID"].apply(lambda x: game_mapping[x])

In [ ]:
Test["Score"].unique()

In [ ]:
submit = Test[["Game_ID", "Score"]]

In [ ]:
submit.shape

In [ ]:
submit.drop_duplicates(subset = ["Game_ID"], inplace=True)

In [ ]:
submit = submit.reset_index(drop=True)

In [ ]:
submit.shape

In [ ]:
submit.to_csv("submission.csv", index=False)